# Predicting winning teams of League of Legends Games

## 1. Introduction

The objective of this League of Legends (LoL) project is to predict the aspects of game plays that influence the winning teams (red or blue) in the game. League of Legends is the most competitive and dominant Multiplayer online battle arena (MOBA) game in the world and as the level of pro players are increased and set to high-quality equalized, importance of data analytics had risen in pro-level tournaments from recent few years.<br><br>
The dataset wasa sourced from the Kaggle at https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min (michel's fanboi, 2020) <br>
The report is organized as follows: <br>
* Section 2 (Overvieew) outlines the methodoloy
* Section 3 (Data Preparation) summarize the data prepareation process and our model evaluation strategy.
* Section 4 (Data Exploration & Visualisation)
* Section 5 (Predictive Modelling)
* Section 6 (Summary & Conclusion)
* Section 7 (Presentation & Table of Contents)

## 2. Overview

### Data Source

The dataset is collected from high-elo solo qued ranked games for first 10 minutes statistics for each team by Michel's fanboi (April, 2020). The dataset consists of 1 csv file, 19 attributes for each teams and 9,879 observations recorded. <br><br>
Baed on our understanding of the game, League of Legends is Player vs Player game, we came up to the point that every variables should be considered relatively. We have considered that how much you play better compare to opponent team is more important rather than objective numbers of how much the certain team does in each game. So we will combine blue and red descriptive features by calculate the differences. 



### Project Objective

Our ultimate goal is predicting winning team based on the information given in first 10 minutes of game plays and identify the most relevant aspects of the game that lead to win games. 

### Target Feature

Our target feature is 'Wins', which 1 if the blue team has won, 0 otherwise.

### Descriptive Features

Each features are relatively based on blue side, differences of blue compare to red.
* WardsPlaced: Number of more warding totems placed by the blue team on the map
* WardsDestroyed: Number of more enemy warding totems the blue team has destroyed
* FirstBlood: First kill of the game. 1 if the blue team did the first kill, 0 otherwise
* Kills: Number of more enemies killed by the team
* Assists: Number of more kill assists
* Dragons: Number of moredragons killed by the blue team
* Heralds: Number of more heralds killed by the blue team
* TowersDestoryed: Number of more structures destroyed by the blue team
* Gold: Differences of the Team total gold
* Experience: Differences of the Team total experience
* MinionsKilled: Differences of the Team total minions killed (CS)
* JungleMinionsKilled: Differences of the Team total jungle monsters killed

Most of the descriptive features are self-explanatory.

## 3. Data Preparation

### Loading Dataset

In [15]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import io
import requests

dataset = pd.read_csv("high_diamond_ranked_10min.csv")
dataset_col = dataset.columns.values

print(f"Shape of the dataset is {dataset.shape} \n")
print(f"Data types are below where 'object' indicates a string type: ")
print(dataset.dtypes)


Shape of the dataset is (9879, 40) 

Data types are below where 'object' indicates a string type: 
gameId                            int64
blueWins                          int64
blueWardsPlaced                   int64
blueWardsDestroyed                int64
blueFirstBlood                    int64
blueKills                         int64
blueDeaths                        int64
blueAssists                       int64
blueEliteMonsters                 int64
blueDragons                       int64
blueHeralds                       int64
blueTowersDestroyed               int64
blueTotalGold                     int64
blueAvgLevel                    float64
blueTotalExperience               int64
blueTotalMinionsKilled            int64
blueTotalJungleMinionsKilled      int64
blueGoldDiff                      int64
blueExperienceDiff                int64
blueCSPerMin                    float64
blueGoldPerMin                  float64
redWardsPlaced                    int64
redWardsDestroyed    

In [25]:
%%capture
!pip install altair vega_datasets
!pip install -e .
!pip install --upgrade altair
!pip install vega vega_datasets
import altair as alt
alt.renderers.enable('notebook')

ValueError: 
To use the 'notebook' renderer, you must install the vega package
and the associated Jupyter extension.
See https://altair-viz.github.io/getting_started/installation.html
for more information.


There are 9879 observations and 1 Target Feature, 38 Descriptive Features and 1 ID column.

### Checking for missing values

In [2]:
print(f"\nNumber of missing values for each feature:")
dataset.isna().sum()


Number of missing values for each feature:


gameId                          0
blueWins                        0
blueWardsPlaced                 0
blueWardsDestroyed              0
blueFirstBlood                  0
blueKills                       0
blueDeaths                      0
blueAssists                     0
blueEliteMonsters               0
blueDragons                     0
blueHeralds                     0
blueTowersDestroyed             0
blueTotalGold                   0
blueAvgLevel                    0
blueTotalExperience             0
blueTotalMinionsKilled          0
blueTotalJungleMinionsKilled    0
blueGoldDiff                    0
blueExperienceDiff              0
blueCSPerMin                    0
blueGoldPerMin                  0
redWardsPlaced                  0
redWardsDestroyed               0
redFirstBlood                   0
redKills                        0
redDeaths                       0
redAssists                      0
redEliteMonsters                0
redDragons                      0
redHeralds    

Dataset provider mentioned that there are no missing values and we confirm again to make sure that there are no missing values.

In [3]:
dataset.sample(n=5, random_state=999)

gameId  blueWins  blueWardsPlaced  blueWardsDestroyed  \
4387  4487157845         0               20                   5   
3500  4525068217         0               16                  25   
790   4524662465         1               16                   1   
7993  4484601181         1               16                   0   
4910  4513962663         0               20                   2   

      blueFirstBlood  blueKills  blueDeaths  blueAssists  blueEliteMonsters  \
4387               1          4           5            7                  0   
3500               0          5           7            5                  0   
790                0         11           4           18                  1   
7993               0          8           5            8                  2   
4910               1          8           7            5                  1   

      blueDragons      ...        redTowersDestroyed  redTotalGold  \
4387            0      ...                         0         16708   
3500            0      ...                         0         17492   
790             1      ...                         0         15759   
7993            1      ...                         0         15674   
4910            0      ...                         0         16805   

      redAvgLevel  redTotalExperience  redTotalMinionsKilled  \
4387          7.4               18842                    254   
3500          7.2               19630                    241   
790           7.0               17401                    205   
7993          6.8               17179                    226   
4910          6.8               18103                    241   

      redTotalJungleMinionsKilled  redGoldDiff  redExperienceDiff  \
4387                           52         1641               1610   
3500                           61          729                620   
790                            68        -3936              -2111   
7993                           36        -1716              -1837   
4910                           48          -10               -223   

      redCSPerMin  redGoldPerMin  
4387         25.4         1670.8  
3500         24.1         1749.2  
790          20.5         1575.9  
7993         22.6         1567.4  
4910         24.1         1680.5  

[5 rows x 40 columns]

### Data Cleaning and Transformation

In [4]:
data = dataset.copy()
data = data.drop(columns=['gameId'])

ID variable is not relevant in our report. <br>

In [5]:
data['Wins'] = data['blueWins']
data['FirstBlood'] = data['blueFirstBlood']
data['WardsPlaced'] = data['blueWardsPlaced'] - data['redWardsPlaced']
data['WardsDestroyed'] = data['blueWardsDestroyed'] - data['redWardsDestroyed']
data['Dragons'] = data['blueDragons'] - data['redDragons']
data['Heralds'] = data['blueHeralds'] - data['redHeralds']
data['Kills'] = data['blueKills'] - data['blueDeaths']
data['Assists'] = data['blueAssists'] - data['redAssists']
data['TowersDestroyed'] = data['blueTowersDestroyed'] - data['redTowersDestroyed']
data['Gold'] = data['blueGoldDiff']
data['Experience'] = data['blueExperienceDiff']
data['MinionsKilled'] = data['blueTotalMinionsKilled'] - data['redTotalMinionsKilled']
data['JungleMinionsKilled'] = data['blueTotalJungleMinionsKilled'] - data['redTotalJungleMinionsKilled']

In [6]:
data = data.drop(columns=['blueWins', 'blueFirstBlood', 'blueWardsPlaced', 'redWardsPlaced', 'blueDragons', 'redDragons', 'blueHeralds', 'redHeralds', 'blueKills', 'redKills', 'blueAssists', 'redAssists', 'blueTowersDestroyed', 'redTowersDestroyed',
                         'blueTotalMinionsKilled', 'redTotalMinionsKilled', 'blueTotalJungleMinionsKilled', 'redTotalJungleMinionsKilled', 'blueWardsDestroyed', 'redWardsDestroyed', 'blueGoldDiff', 'blueExperienceDiff'])

We combine blue and red features into one to calculate the differences. 

In [7]:
data = data.drop(columns=['redDeaths', 'redFirstBlood', 'blueEliteMonsters', 'redEliteMonsters', 'blueDeaths', 'blueCSPerMin', 'blueGoldPerMin', 'redCSPerMin', 'redGoldPerMin', 'redGoldDiff', 'redExperienceDiff',
                          'blueTotalExperience', 'redTotalExperience', 'blueTotalGold', 'redTotalGold', 'blueAvgLevel', 'redAvgLevel'])

We drop descriptive features that duplicate the existing features which contain similar or duplicated information.
* Deaths: Relevant deaths can be found from each team's kills.
* redFirstblood: Firstblood is either blue or red. We use blueFirstblood.
* EliteMonsters: Sum of dragons and heralds.
* AvgLevel: Duplicate of TotalExperience
* CSPerMin: Duplicate of TotalMinionsKilled
* GoldPerMin: Duplicate of TotalGold
* TotalExperience: We use ExperienceDiff.
* TotalGold: We use GoldDiff

 

In [8]:
data.sample(n=5, random_state=999)

Wins  FirstBlood  WardsPlaced  WardsDestroyed  Dragons  Heralds  Kills  \
4387     0           1            1               1        0        0     -1   
3500     0           0          -27              20       -1       -1     -2   
790      1           0            4              -1        1       -1      7   
7993     1           0            1              -1        1        1      3   
4910     0           1            1              -4        0        1      1   

      Assists  TowersDestroyed  Gold  Experience  MinionsKilled  \
4387       -3                0 -1641       -1610            -82   
3500       -1                0  -729        -620             11   
790        14                0  3936        2111             53   
7993        0                0  1716        1837            -14   
4910       -3                0    10         223            -34   

      JungleMinionsKilled  
4387                    4  
3500                  -13  
790                   -20  
7993                   24  
4910                   -7

### Summary statistics

In [9]:
data.describe(include='all')

Wins   FirstBlood  WardsPlaced  WardsDestroyed      Dragons  \
count  9879.000000  9879.000000  9879.000000     9879.000000  9879.000000   
mean      0.499038     0.504808    -0.079664        0.101731    -0.051119   
std       0.500024     0.500002    25.960582        2.854910     0.878945   
min       0.000000     0.000000  -260.000000      -23.000000    -1.000000   
25%       0.000000     0.000000    -4.000000       -1.000000    -1.000000   
50%       0.000000     1.000000     0.000000        0.000000     0.000000   
75%       1.000000     1.000000     4.000000        1.000000     1.000000   
max       1.000000     1.000000   234.000000       23.000000     1.000000   

           Heralds        Kills      Assists  TowersDestroyed          Gold  \
count  9879.000000  9879.000000  9879.000000      9879.000000   9879.000000   
mean      0.027938     0.046260    -0.017006         0.008402     14.414111   
std       0.589292     4.195489     5.766787         0.324835   2453.349179   
min      -1.000000   -17.000000   -25.000000        -2.000000 -10830.000000   
25%       0.000000    -3.000000    -4.000000         0.000000  -1585.500000   
50%       0.000000     0.000000     0.000000         0.000000     14.000000   
75%       0.000000     3.000000     4.000000         0.000000   1596.000000   
max       1.000000    16.000000    23.000000         4.000000  11467.000000   

        Experience  MinionsKilled  JungleMinionsKilled  
count  9879.000000    9879.000000          9879.000000  
mean    -33.620306      -0.649661            -0.803421  
std    1920.370438      30.942675            14.274733  
min   -9333.000000    -120.000000           -72.000000  
25%   -1290.500000     -21.000000           -10.000000  
50%     -28.000000      -1.000000             0.000000  
75%    1212.000000      20.000000             8.000000  
max    8348.000000     127.000000            64.000000

### Scailing Features

In [10]:
from sklearn import preprocessing

data_df = data.copy()
data_scaler = preprocessing.MinMaxScaler()
data_scaler.fit(data)
data = data_scaler.fit_transform(data)

pd.DataFrame(data, columns=data_df.columns).sample(5, random_state=999)

Wins  FirstBlood  WardsPlaced  WardsDestroyed  Dragons  Heralds  \
4387   0.0         1.0     0.528340        0.521739      0.5      0.5   
3500   0.0         0.0     0.471660        0.934783      0.0      0.0   
790    1.0         0.0     0.534413        0.478261      1.0      0.0   
7993   1.0         0.0     0.528340        0.478261      1.0      1.0   
4910   0.0         1.0     0.528340        0.413043      0.5      1.0   

         Kills   Assists  TowersDestroyed      Gold  Experience  \
4387  0.484848  0.458333         0.333333  0.412118    0.436797   
3500  0.454545  0.500000         0.333333  0.453021    0.492789   
790   0.727273  0.812500         0.333333  0.662242    0.647248   
7993  0.606061  0.520833         0.333333  0.562677    0.631752   
4910  0.545455  0.458333         0.333333  0.486164    0.540467   

      MinionsKilled  JungleMinionsKilled  
4387       0.153846             0.558824  
3500       0.530364             0.433824  
790        0.700405             0.382353  
7993       0.429150             0.705882  
4910       0.348178             0.477941

### Feature Selection and Ranking

In [22]:
from sklearn.ensemble import RandomForestClassifier

target = data_df['Wins']
num_features = 12
model_rfi = RandomForestClassifier(n_estimators=100)
model_rfi.fit(data, target)
fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:num_features]

best_features_rfi = data_df.columns[fs_indices_rfi].values
feature_importances_rfi = model_rfi.feature_importances_[fs_indices_rfi]

%config InlineBackend.figure_format = 'retina'

import altair as alt
alt.renderers.enable('notebook') # required in Jupyter notebooks

def plot_imp(best_features, scores, method_name, color):
    
    df = pd.DataFrame({'features': best_features, 
                       'importances': scores})
    
    chart = alt.Chart(df, 
                      width=500, 
                      title=method_name + ' Feature Importances'
                     ).mark_bar(opacity=0.85, 
                                color=color).encode(
        alt.X('features', title='Feature', sort=None, axis=alt.AxisConfig(labelAngle=45)),
        alt.Y('importances', title='Importance')
    )
    
    return chart

ValueError: 
To use the 'notebook' renderer, you must install the vega package
and the associated Jupyter extension.
See https://altair-viz.github.io/getting_started/installation.html
for more information.


In [ ]:
plot_imp(best_features_rfi, feature_importances_rfi, 'Random Forest', 'blue')